In [17]:
from pathlib import Path
from cryptography import x509
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from bs4 import BeautifulSoup
import requests
import csv
from datetime import datetime

In [18]:
extentions = [".pem"]
root_path = '/etc/ssl/certs'

In [19]:
blacklist_cert_list=[]

In [20]:
def loadBlacklistedSSls():
    CSV_URL="https://sslbl.abuse.ch/blacklist/sslblacklist.csv"

    download = requests.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    lines = decoded_content.splitlines()
    cr = csv.reader(lines, delimiter=',')
    my_list = list(cr)
    headers = None
    results = []
    for row in my_list:
        if len(row) == 3:
            if headers is None:
                headers = row
                headers[0]=headers[0].strip('#').strip()
                continue
            results.append(dict(zip(headers,row)))
    return results
blacklist_cert_list += loadBlacklistedSSls()
blacklist_cert_list[-1]

In [21]:
print("Please review the list and make sure you do not have any banned certs")
for item in Path(root_path).iterdir():
    if item.is_file():
        if item.suffix in extentions:
            cert = x509.load_pem_x509_certificate(item.read_bytes(), default_backend())
            sha1_hashbytes = cert.fingerprint(hashes.SHA1())
            sha1_spaces= " ".join([format(i,'02x') for i in sha1_hashbytes])
            sha1 = "".join([format(i,'02x') for i in sha1_hashbytes])
            sha256_hashbytes = cert.fingerprint(hashes.SHA256())
            sha256 = " ".join([format(i,'02x') for i in sha256_hashbytes])
            trusted = False
            for b in blacklist_cert_list:
                if 'SHA1' in b and b['SHA1'] == sha1:
                    print
                    print("###### BANNED CERT:",b['Listingreason']) 
                    print('Filename:',item.stem)
                    print('Issuer:',cert.issuer)
                    print('Subject:',cert.subject)
                    print('Expires After:',cert.not_valid_after)
                    print('SN:',cert.serial_number)
                    print(sha1)
                    print(sha256)
print('>finished')

In [22]:
trusted_ca_list = []

In [23]:
def loadVisaTrustedCAs():
    url = "https://developer.visa.com/pages/trusted_certifying_authorities"
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    table = soup.find('table')
    if table is None:
        raise Exception(f"Table missing please check: {url}")
    first = True
    results = []
    for row in table.find_all('tr'):
        if first:
            first = False
            continue
        tds = row.find_all('td')
        results.append({
            'CA':tds[0].text.strip(),
            'Root Certificate Types':tds[1].text.strip(),
            'SHA1-Thumbprint':tds[2].text.strip().strip('\u200e'),
            'src':"Visa"
        })
    return results
trusted_ca_list += loadVisaTrustedCAs()
trusted_ca_list[-1]

In [24]:
def loadMicrosoftTrustedCAs():
    CSV_URL="https://ccadb-public.secure.force.com/microsoft/IncludedCACertificateReportForMSFTCSV"

    download = requests.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    lines = decoded_content.splitlines()
    cr = csv.reader(lines, delimiter=',')
    my_list = list(cr)
    headers = None
    results = []
    for row in my_list:
        
        if len(row) == 10:
            if headers is None:
                headers = row
                headers[0]=headers[0].strip('#').strip()
                continue
            data = dict(zip(headers,row))
            data['src']='Microsoft'
            results.append(data)
    return results
trusted_ca_list += loadMicrosoftTrustedCAs()
trusted_ca_list[-1]

In [25]:
def loadAppleTrustedCAs():
    url="https://support.apple.com/en-us/HT212773#trusted"
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    table = soup.find('table')
    if table is None:
        raise Exception(f"Table missing please check: {url}")
    first = True
    results = []
    for row in table.find_all('tr'):
        if first:
            first = False
            continue
        tds = row.find_all('td')
        results.append({
            'CA':tds[0].text.strip(),
            'Root Certificate Types':tds[2].text.strip(),
            'SHA256-Thumbprint':tds[8].text.strip(),
            "SRC":"Apple"
        })
    return results
trusted_ca_list += loadAppleTrustedCAs()

In [26]:
print('Certs listed below are expired')
print()
today = datetime.now()
for item in Path(root_path).iterdir():
    if item.is_file():
        if item.suffix in extentions:
            cert = x509.load_pem_x509_certificate(item.read_bytes(), default_backend())
            
            if(cert.not_valid_after<today):
                print('Expired Cert', cert)
                print('Filename:',item.stem)
                print('Issuer:',cert.issuer)
                print('Subject:',cert.subject)
                print('Expires After:',cert.not_valid_after)
                print('SN:',cert.serial_number)
                print(sha1)
                print(sha256)
                print()
print('>finished')

In [28]:
print('Please verify any certs listed below')
print()
for item in Path(root_path).iterdir():
    if item.is_file():
        if item.suffix in extentions:
            cert = x509.load_pem_x509_certificate(item.read_bytes(), default_backend())
            sha1_hashbytes = cert.fingerprint(hashes.SHA1())
            sha1_spaces= " ".join([format(i,'02x') for i in sha1_hashbytes])
            sha1 = "".join([format(i,'02x') for i in sha1_hashbytes])
            sha256_hashbytes = cert.fingerprint(hashes.SHA256())
            sha256_spaces = " ".join([format(i,'02x') for i in sha256_hashbytes])
            sha256 = "".join([format(i,'02x') for i in sha256_hashbytes])
            trusted = False
            for t in trusted_ca_list:
                if'SHA1-Thumbprint' in t:
                    if t['SHA1-Thumbprint'] == sha1_spaces:
                        trusted = True
                        break
                    else:
                        issue=f"SHA1-Thumbprint {sha1_spaces} != {t['SHA1-Thumbprint']}"
                        break
                        
                if'SHA256-Thumbprint' in t:
                    if t['SHA256-Thumbprint'] == sha256_spaces:
                        trusted = True
                        break
                    else:
                        issue=f"SHA256-Thumbprint {sha256_spaces} != {t['SHA256-Thumbprint']}"
                        break
                        
                if 'SHA-1 Fingerprint' in t:
                    if t['SHA-1 Fingerprint'] == sha1.upper():
                        trusted = True
                        break
                    else:
                        issue=f"SHA-1 Fingerprint {sha1.upper()} != {t['SHA-1 Fingerprint']}"
                        break
                        
                if 'SHA-256 Fingerprint' in t:
                    if t['SHA-256 Fingerprint'] == sha256.upper():
                        trusted = True
                        break
                    else:
                        issue=f"SHA-256 Fingerprint {sha256.upper()} != {t['SHA-256 Fingerprint']}"
                        break
                        
                
            if trusted == False:
                print('Filename:',item.stem)
                print('Issuer:',cert.issuer)
                print('Subject:',cert.subject)
                print('Expires After:',cert.not_valid_after)
                print('SN:',cert.serial_number)
                print('Issue:',issue)
                print("Trust By", t['src'])
                print(sha1)
                print(sha256)
                print()
print('>finished')